In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/genre-classification-dataset-imdb/Genre Classification Dataset/description.txt
/kaggle/input/genre-classification-dataset-imdb/Genre Classification Dataset/test_data_solution.txt
/kaggle/input/genre-classification-dataset-imdb/Genre Classification Dataset/test_data.txt
/kaggle/input/genre-classification-dataset-imdb/Genre Classification Dataset/train_data.txt


In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.regularizers import l2
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam

2024-03-14 10:11:12.733369: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-14 10:11:12.733487: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-14 10:11:12.738568: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score

In [4]:
df = pd.read_csv("/kaggle/input/genre-classification-dataset-imdb/Genre Classification Dataset/train_data.txt",sep=':::', names=['ID', 'TITLE', 'GENRE', 'DESCRIPTION'])
display(df.head())
print(df.shape)

/tmp/ipykernel_303/3152433989.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv("/kaggle/input/genre-classification-dataset-imdb/Genre Classification Dataset/train_data.txt",sep=':::', names=['ID', 'TITLE', 'GENRE', 'DESCRIPTION'])


,ID,TITLE,GENRE,DESCRIPTION
0,1,Oscar et la dame rose (2009),drama,Listening in to a conversation between his do...
1,2,Cupid (1997),thriller,A brother and sister with a past incestuous r...
2,3,"Young, Wild and Wonderful (1980)",adult,As the bus empties the students for their fie...
3,4,The Secret Sin (1915),drama,To help their unemployed father make ends mee...
4,5,The Unrecovered (2007),drama,The film's title refers not only to the un-re...


(54214, 4)


import re
def clean_text(text):
    # Remove special characters, punctuation, and symbols
    text = re.sub(r'[^\w\s]', '', text)
    return text

In [5]:
df[0:1]

,ID,TITLE,GENRE,DESCRIPTION
0,1,Oscar et la dame rose (2009),drama,Listening in to a conversation between his do...


In [6]:
df.drop(columns=['ID','TITLE'],inplace=True)

In [7]:
df.head()

,GENRE,DESCRIPTION
0,drama,Listening in to a conversation between his do...
1,thriller,A brother and sister with a past incestuous r...
2,adult,As the bus empties the students for their fie...
3,drama,To help their unemployed father make ends mee...
4,drama,The film's title refers not only to the un-re...


In [8]:
average_length = df['DESCRIPTION'].str.len().mean()

print(f"The average length of movie descriptions is: {average_length:.2f}")


The average length of movie descriptions is: 600.45


In [10]:
X=df['DESCRIPTION']
y=df['GENRE']

In [11]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [12]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [15]:
import re
def clean_text(text):
    # Remove special characters, punctuation, and symbols
    text = re.sub(r'[^\w\s]', '', text)
    return text
X_train=X_train.apply(clean_text)
X_test=X_test.apply(clean_text)

In [16]:
from nltk.corpus import stopwords

def remove_stopwords(text):
    stop_words = set(stopwords.words('english'))
    new_text = [word for word in text.split() if word not in stop_words]
    return ' '.join(new_text)
X_train=X_train.apply(remove_stopwords)
X_test=X_test.apply(remove_stopwords)

In [17]:
import spacy 
nlp=spacy.load('en_core_web_sm')
import re 
def tokenize(text):
    tokens=re.split('\W+',text)
    return tokens
X_train=X_train.apply(tokenize)
X_test=X_test.apply(tokenize)

In [18]:
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()
def steam_words(text):
    return " ".join([ps.stem(word) for word in text])
X_train=X_train.apply(steam_words)
X_test=X_test.apply(steam_words)

In [21]:
X_train

19253    s e x   b e t r a y   s e d u c t   m a n i p ...
12261    a u t i s t   t e e n   t o n i   f r i e n d ...
37143    t h e   o n e h o u r   s p e c i a l   e x a ...
21965    t h i   p e r s o n   n a r r   d o c u m e n ...
7033     t h e   e a s y g o   v i l l a g   p o l i c ...
                               ...                        
11284    d a v i d   c r o n e n b e r g   w e l l k n ...
44732    a d a n   n e u m a n n   m e m b e r   j e w ...
38158    m i r a n d a   l e s b i a n   p h o t o g r ...
860      t h e y   e n e m i   p r o f e s s i o n   l ...
15795    I N   2 0 1 5   f a s h i o n   d e s i g n   ...
Name: DESCRIPTION, Length: 43371, dtype: object

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=100)  
X_train_vectorized = vectorizer.fit_transform(X_train).toarray()
print(X_train_vectorized)
X_test_vectorized = vectorizer.transform(X_test)

ValueError: empty vocabulary; perhaps the documents only contain stop words

In [ ]:
print(tfidf)
print(tfidf.get_feature_names_out())

In [ ]:
df['DESCRIPTION_1']

In [ ]:
X.head()

In [ ]:
y.head()

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
import pandas as pd
import re
import numpy as np

In [ ]:
print(f"The shpae of X_train is {X_train.shape}")
print(f"The shpae of X_test is {X_test.shape}")
print(f"The shpae of Y_train is {y_train.shape}")
print(f"The shpae of Y_test is {y_test.shape}")

In [ ]:
vectorizer = TfidfVectorizer(max_features=1000)  
X_train_vectorized = vectorizer.fit_transform(X_train).toarray()
print(X_train_vectorized)
X_test_vectorized = vectorizer.transform(X_test)

In [ ]:
# Convert target labels to numerical values
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

In [ ]:
y_train_encoded

In [ ]:
print(X_train_vectorized.shape)


In [ ]:
X_train_vectorized

In [ ]:
# X_train_vectorized_ordered = tf.sparse.reorder(X_train_vectorized)

In [ ]:
# Convert them to SparseTensor
X_train_sparse = tf.sparse.from_dense(X_train_vectorized)
X_test_sparse = tf.sparse.from_dense(X_test_vectorized)

# Now reorder the SparseTensors
X_train_sparse = tf.sparse.reorder(X_train_sparse)
X_test_sparse = tf.sparse.reorder(X_test_sparse)

In [ ]:
model = Sequential([
    Dense(512, activation='relu', input_dim=X_train_vectorized.shape[1]),
    Dropout(0.8),
    Dense(256, activation='relu'),
    # Dropout(0.8),
    Dense(128, activation='relu'),
    # Dropout(0.8),

    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(27, activation='softmax')
])



optimizer = Adam(learning_rate=0.003)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train_vectorized, y_train_encoded, epochs=15, batch_size=64, validation_data=(X_test_vectorized, y_test_encoded))
# Evaluate the model on the testing dataset
test_loss, test_accuracy = model.evaluate(X_test_vectorized, y_test_encoded)

print(f'Test Accuracy: {test_accuracy}')
print(f'Test Loss: {test_loss}')